## Fake NEws Classificer USsing Bidirectional LSTM

In [2]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd



In [4]:
# df=pd.read_csv('train.csv')
import pandas as pd

# Try specifying the encoding explicitly and skip bad lines
df = pd.read_csv('/content/train.csv', engine='python', on_bad_lines='skip')

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [5]:
df.isnull().sum()

,0
id,0
title,366
author,1289
text,45
label,16


In [6]:
df.shape

(13594, 5)

In [7]:
df=df.dropna()

In [8]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [9]:
## get the independent features
X=df.drop('label',axis=1)

In [10]:
y=df['label']

In [11]:
## Check whether dataset is balanced or not
y.value_counts()

,count
label,
0.0,6772
1.0,5169


In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.17.0'

In [14]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [15]:
## vocabulary size
voc_size=5000

In [16]:
messages=X.copy()

In [17]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
messages.reset_index(inplace=True)

In [22]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [25]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4211, 4406, 4537, 9, 3295, 3954, 2679, 380, 2575, 1037],
 [1697, 987, 875, 1374, 2722, 1737, 236],
 [4986, 2653, 3276, 4402],
 [4182, 1280, 4703, 527, 4369, 4922],
 [555, 2722, 1968, 1499, 4069, 775, 2722, 32, 3701, 2203],
 [3003,
  1924,
  1325,
  388,
  2336,
  4306,
  2889,
  1410,
  1329,
  4825,
  4539,
  477,
  3887,
  1627,
  236],
 [4698, 4628, 4559, 864, 3766, 1311, 3767, 821, 319, 2277, 3080],
 [1389, 295, 985, 2136, 3676, 4168, 4306, 2914, 319, 2277, 3080],
 [754, 2269, 1652, 4211, 2622, 244, 4570, 3309, 4306, 2794],
 [3908, 4393, 1628, 480, 703, 4003, 1162, 1109],
 [4230, 4078, 4612, 3480, 2043, 3178, 2179, 169, 635, 4277, 276],
 [527, 3680, 3295, 244, 4306, 3676],
 [4954, 2673, 4899, 1015, 1569, 3669, 3752, 1932, 2836],
 [2518, 1459, 3080, 1420, 1429, 1131, 264, 319, 2277, 3080],
 [54, 765, 221, 3980, 3877, 319, 2277, 3080],
 [1864, 3002, 4643, 2448, 2511, 730, 2866, 1939, 1907, 1566],
 [805, 2339, 987],
 [1198, 3295, 3493, 2548, 4306, 654, 1935, 236],
 [4660, 3981, 875,

## Embedding Representation

In [26]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ...,  380, 2575, 1037],
       [   0,    0,    0, ..., 2722, 1737,  236],
       [   0,    0,    0, ..., 2653, 3276, 4402],
       ...,
       [   0,    0,    0, ...,  319, 2277, 3080],
       [   0,    0,    0, ...,  700, 3248, 1160],
       [   0,    0,    0, ..., 4061, 4230, 4899]], dtype=int32)

In [27]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [30]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.8030 - loss: 0.4036 - val_accuracy: 0.9173 - val_loss: 0.2180
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.9476 - loss: 0.1391 - val_accuracy: 0.9170 - val_loss: 0.1944
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9752 - loss: 0.0713 - val_accuracy: 0.9137 - val_loss: 0.2564
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9846 - loss: 0.0439 - val_accuracy: 0.9054 - val_loss: 0.3493
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.9928 - loss: 0.0257 - val_accuracy: 0.9122 - val_loss: 0.3407
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.9940 - loss: 0.0212 - val_accuracy: 0.8993 - val_loss: 0.4646
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9965 - loss: 0.0113 - val_accuracy: 0.9048 - val_loss: 0.4972
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.9968 - loss: 0.0100 - val_accuracy: 0

In [31]:
y_pred=model.predict(X_test)

124/124 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [32]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [33]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[2058,  163],
       [ 251, 1469]])

In [34]:
print(accuracy_score(y_test,y_pred))

0.8949505201725451
